In [2]:
from __future__ import print_function
import tensorflow as tf

In [3]:
# Make 'constant' nodes: no input, constant output.

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
# To get output must run session

sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [5]:
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [6]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = tf.add(a,b)

In [8]:
subtractor_node = tf.subtract(a,b)
sess.run(subtractor_node,{a:5,b:2})

3.0

In [10]:
sess.run(tf.multiply(a,b),{a:50,b:3})

150.0

In [9]:
# adding 1 dimensional outputs
print(sess.run(adder_node, {a: 3, b: 4.5}))

7.5


In [8]:
# NB don't need to pre-define adder node, but a and b must be already
print(sess.run(tf.add(a,b), {a: 3, b: 4.5}))

7.5


In [9]:
# adding 2 dimensional outputs 
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

[ 3.  7.]


In [10]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In [18]:
c = tf.placeholder(tf.float32)
multiplier_node = adder_node*cw

In [19]:
print(sess.run(multiplier_node,{a:5,b:2, c:3}))

21.0


In [11]:
# Variables allow arbitary inputs. 
#Are not initialised when tf.Variable is called
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

In [12]:
# Initialising all var's in TF program
init = tf.global_variables_initializer()
sess.run(init)

In [13]:
# x is a placeholder; can eval linear_model for several values
# get the previously-initialised tf session to run this, with some inputs
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [14]:
# make placeholder for true labels. 
y = tf.placeholder(tf.float32)
# define a function using previously defined 'linear model' function and y variable
squared_deltas = tf.square(linear_model - y)
# tf function that takes an input tensor
loss = tf.reduce_sum(squared_deltas) 
# get the previously-initialised tf session to run this, with some inputs
# ie takes x, does linear_model on this. then squared_deltas on its output and y. Then loss on that.
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [15]:
# tf.Variable initialises tf variables.
# tf.assign takes two arguments: name and value, and reassigns the value to that name
# this then is run by the session
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


Training optimisers to minimise loss function - compute deriv's using tf.gradients

In [27]:
optimizer = tf.train.GradientDescentOptimizer(0.01)

# loss is calculated from squared_deltas, which uses linear model and x and y
# .minimize does one iteration of reduction
train = optimizer.minimize(loss)

10 iterations... each time, run 'train' on feature values x and outputs y.

'train' minimises 'loss' which is calculated from 'squared deltas'

..which is calculated from linear model on x, and y

In [28]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
#   print (sess.run([W, b]))
    
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## Complete program

In [29]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


# Tensorflow estimator

### high level library runnign training/eval loops, manage datasets

In [30]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_service': None, '_is_chief': True, '_save_summary_steps': 100, '_task_id': 0, '_task_type': 'worker', '_num_worker_replicas': 1, '_model_dir': '/var/folders/rf/hjjjk251633cl8pln8zlkl5w000cm7/T/tmpc4dvkofn', '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11ba4ff98>, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/rf/hjjjk251633cl8pln8zlkl5w000cm7/T/tmpc4dvkofn/model.ckpt.
INFO:tensorflow:loss = 13.0, step = 1
INFO:tensorflow:global_step/sec: 935.692
INFO:tensorflow:loss = 0.181613, step = 101 (0.125 sec)
INFO:tensorflow:global_step/sec: 971.129
INFO:tensorflow:loss = 0.0900655, step = 

In [20]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_num_worker_replicas': 1, '_master': '', '_save_checkpoints_steps': None, '_service': None, '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11b148898>, '_task_type': 'worker', '_is_chief': True, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/rf/hjjjk251633cl8pln8zlkl5w000cm7/T/tmpcvf_3v1p', '_log_step_count_steps': 100, '_save_summary_steps': 100, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/rf/hjjjk251633cl8pln8zlkl5w000cm7/T/tmpcvf_3v1p/model.ckpt.
INFO:tensorflow:loss = 28.7265313042, step = 1
INFO:tensorflow:global_step/sec: 1083.93
INFO:tensorflow:loss = 0.0681124619016, step = 101 (0.095 sec)
INFO:tensorflow:global_step/sec: 1172.82
INFO:tensorflow:loss = 0.